* Course: YSC4103 MCS Capstone
* Date created: 2019/02/25
* Name: Linfan XIAO
* Description: Implement the Fokas method as described in "Evolution PDEs and augmented eigenfunctions. Finite interval."



# Importing packages

In [120]:
using SymPy
using PyCall
sympy = pyimport("sympy")
# using Roots
using Distributions
# using IntervalArithmetic
# using IntervalRootFinding
using ApproxFun

# Global variables

## `tol`

Error tolerance level.

In [2]:
tol = 1e-05

1.0e-5

## `signif`

The number of significant digits to display in symbolic expressions.

In [3]:
signif = 3

3

## `infty`

A number representing infinity in numerical approximations.

In [4]:
infty = 10

10

# Helper functions

## `check_all(array, condition)`

Checks whether all elements in an array satisfy a given condition.

In [5]:
function check_all(array, condition)
    for x in array
        if !condition(x)
            return false
        end
    end
    return true
end

check_all (generic function with 1 method)

**Parameters**
* `array`: `Array`
    * Input array to be checked. Generic, not necessarily homogeneous.
* `condition`: `Function: Bool`
    * Boolean function to be applied to each element in the array.

**Returns**
* `check_all`: `Bool`
    * Returns `true` if all elements in `array` satisfy `condition` and `false` if any element does not satisfy the condition.
    
**Example**

In [6]:
array = [0,1,2,3]
condition = x -> x>2
check_all(array, condition)

false

## `check_any(array, condition)`

Checks whether any element in an array satisfy a given condition.

In [7]:
function check_any(array, condition)
    for x in array
        if condition(x)
            return true
        end
    end
    return false
end

check_any (generic function with 1 method)

**Parameters**
* `array`: `Array`
    * Input array to be checked. Generic, not necessarily homogeneous.
* `condition`: `Function: Bool`
    * Boolean function to be applied to each element in the array.

**Returns**
* `check_all`: `Bool`
    * Returns `true` if there exists an element in the array that satisfies a given condition and `false` if no element satisfies the condition.
    
**Example**

In [8]:
array = [0,1,2,3]
condition = x -> x>2
check_any(array, condition)

true

## `set_tol(x, y)`

Sets an appropriate tolerance for checking whether two numbers are approximately equal.

In [9]:
function set_tol(x::Number, y::Number)
    return tol * mean([x y]) # tol is global variable
end

set_tol (generic function with 1 method)

**Parameters**
* `x`, `y`: `Number`
    * Numbers to compare.

**Returns**
* `set_tol`: `Number`
    * Returns a tolerance within which `x` and `y` are considered approximately equal.

**Example**

In [10]:
x = 14
y = 21
set_tol(x,y)

0.00017500000000000003

## `set_tolMatrix(A, B)`

Sets an appropriate tolerance for checking whether two matrices are approximately equal element-wise.

In [11]:
function set_tolMatrix(A::Array, B::Array)
    if size(A) != size(B)
        throw(error("Matrix dimensions do not match"))
    end
    # Avoid InexactError() when taking norm()
    A = convert(Array{Complex}, A)
    B = convert(Array{Complex}, B)
    return tol * (norm(A,2) + norm(B,2))
end

set_tolMatrix (generic function with 1 method)

**Parameters**
* `A`, `B`: `Array`
    * Numeric matrices to compare.

**Returns**
* `set_tolMatrix`: `Number`
    * Returns a number within which `A` and `B` are considered approximately equal element-wise.

**Example**

In [12]:
A = [4 0.6; 3 2]
B = [5 1; 10 3]
set_tolMatrix(A, B)

0.00016901192145623727

## `evaluate(func, x, t=nothing)`

Evaluate a function at a given value.

In [13]:
function evaluate(func::Union{Function,Number}, x::Number, t = nothing)
    if isa(func, Function)
        return func(x)
    elseif isa(func, SymPy.Sym) # SymPy.Sym must come before Number because SymPy.Sym will be recognized as Number
        if isa(t, Void)
            throw("t required")
        end
        return subs(func, t, x)
    else
        return func
    end
end

evaluate (generic function with 2 methods)

**Parameters**
* `func`: `Function`, `SymPy.Sym`, or `Number`
    * Function to be evaluated. Note that `SymPy.Sym` is absorbed into `Number`.
* `x`: `Number`
    * Value on which `func` is to be evaluated at.
* `t`: `SymPy.Sym` 
    * Free symbol in `func` if `func` is a `SymPy.Sym` object.

**Returns**
* `evaluate`: `Number`
    * Returns the value of `func` at `x`.

**Example**

In [14]:
func = x -> x+1
x = 2
evaluate(func, x)

3

## `partition(n)`

Generate ordered two-integer partitions ($0$ included) of a given non-negative integer.

In [15]:
function partition(n::Int)
    if n < 0
        throw("Non-negative n required")
    end
    output = []
    for i = 0:n
        j = n - i
        push!(output, (i,j))
    end
    return output
end

partition (generic function with 1 method)

**Parameters**
* `n`: `Int`
    * Non-negative integer to be partitioned.

**Returns**
* `partition`: `Array` of `Tuple{Int, Int}`
    * Returns an array of tuples, where each tuple corresponds to a two-integer parition of `n`. Note that a tuple is ordered, and `(i, j)` and `(j, i)` are considered distinct partitions.

**Example**

In [16]:
n = 5
partition(5)

6-element Array{Any,1}:
 (0, 5)
 (1, 4)
 (2, 3)
 (3, 2)
 (4, 1)
 (5, 0)

## `get_symDeriv(u, t, k)`

Constructs the symbolic expression for the $k$th derivative of a univariate function.

In [17]:
function get_symDeriv(u::SymPy.Sym, t::SymPy.Sym, k::Int)
    if k < 0
        throw(error("Non-negative k required"))
    end
    y = u
    for i = 1:k
        newY = diff(y, t)
        y = newY
    end
    return y
end

get_symDeriv (generic function with 1 method)

**Parameters**
* `u`: `SymPy.Sym` 
    * Symbolic expression in `t` to be differentiated.
* `t`: `SymPy.Sym`
    * Free symbol in `u`.
* `k`: `Int`
    * Degree of the desired derivative.

**Returns**
* `get_symDeriv`: `SymPy.Sym`
    * Returns the symbolic expression of $\frac{d^k}{dt^k}u(t)$.
    
**Example**

In [18]:
t = symbols("t")
u = t^2+2t+3
get_symDeriv(u, t, 1)

2*t + 2

## `add_func(f, g)`

Computes the sum of two functions using the function addition given by $(f + g)(x) := f(x) + g(x)$.

In [19]:
function add_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f + g
            else
                return f + g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) + g
            else
                return f(x) + g(x)
            end
        end
    end
    return h
end

add_func (generic function with 1 method)

**Parameters**
* `f`, `g`: `Function` or `Number`
    * Functions to be added.

**Returns**
* `add_func`: `Function` or `Number`
    * Returns the sum of `f` and `g`.

**Example**

In [20]:
f(x) = x^3+1
g(x) = 4x
x = 2
add_func(f, g)(x) == f(x) + g(x)

true

## `mult_func(f, g)`

Computes the product of two functions using the function multiplication given by $(f \cdot g)(x) := f(x) \cdot g(x)$.

In [21]:
function mult_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f * g
            else
                return f * g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) * g
            else
                return f(x) * g(x)
            end
        end
    end
    return h
end

mult_func (generic function with 1 method)

**Parameters**
* `f`, `g`: `Function` or `Number`
    * Functions to be multiplied.

**Returns**
* `add_func`: `Function` or `Number`
    * Returns the product of `f` and `g`.

**Example**

In [22]:
f(x) = x^3+1
g(x) = 4x
x = 2
mult_func(f, g)(x) == f(x) * g(x)

true

## `evaluate_matrix(matrix, a, t=nothing)`

Evaluates a matrix of univariate functions at a given value.

In [23]:
function evaluate_matrix(matrix::Array, a::Number, t = nothing)
    (m, n) = size(matrix)
    matrixA = Array{Number}(m,n)
    for i = 1:m
        for j = 1:n
            matrixA[i,j] = evaluate(matrix[i,j], a, t)
        end
    end
    return matrixA
end

evaluate_matrix (generic function with 2 methods)

**Parameters**
* `matrix`: `Array` of `Function`, `SymPy.Sym`, or `Number`
    * Matrix of univariate functions to evaluate.
* `a`: `Number`
    * Number at which `matrix` is to be evaluated.
* `t`: `SymPy.Sym`
    * Free symbol in the functions in `matrix` if they are `SymPy.Sym` objects.

**Returns**
* `evaluate_matrix`: `Array`
    * Returns a matrix of numbers that correspond to the evalauted functions.

**Example**

In [24]:
matrix = [x->2x x->1; x->x^3 x-> x]
a = 1
evaluate_matrix(matrix, a)

2×2 Array{Number,2}:
 2  1
 1  1

## `get_polynomial(coeffList)`

Given the array $[a_n, a_{n-1}, \ldots, a_1, a_0]$, constructs the polynomial 
$$a_nx^n + a_{n-1}x^{n-1} + \cdots + a_1 x + a_0.$$

In [25]:
function get_polynomial(coeffList::Array)
    polynomial = 0
    n = length(coeffList)-1
    for i in 0:n
        newTerm = t -> coeffList[i+1] * t^(n-i)
        polynomial = add_func(polynomial, newTerm)
    end
    return polynomial
end

get_polynomial (generic function with 1 method)

**Parameters**
* `coeffList`: `Array` of `Number`
    * Array of coefficients of $x^n, x^{n-1}, \ldots, x, 1$ in the polynomial.

**Returns**
* `get_polynomial: Function`
    * Returns a polynomial whose coefficients are given by `coeffList`.
 
**Example**

In [26]:
coeffList = [0,1,2,3]
x = 5
get_polynomial(coeffList)(x) == 0*x^3 + 1*x^2 + 2*x^1 + 3*x^0

true

## `get_polynomialDeriv(coeffList, k)`

Computes the $k$th derivative of a polynomial with known coefficients.

In [27]:
function get_polynomialDeriv(coeffList::Array, k::Int)
    if k < 0
        throw(error("Only nonnegative degrees are allowed"))
    elseif k == 0
        newCoeffList = coeffList
    else
        for counter = 1:k
            n = length(coeffList)
            newCoeffList = hcat([0],[(n-i)*coeffList[i] for i in 1:(n-1)]')
            coeffList = newCoeffList
        end
    end
    return get_polynomial(newCoeffList)
end

get_polynomialDeriv (generic function with 1 method)

**Parameters**
* `coeffList`: `Array` of `Number` 
    * Array of coefficients of $x^n, x^{n-1}, \ldots, x, 1$ in the polynomial.
* `k`: `Int` 
    * Degree of the desired derivative.

**Returns**
* `get_polynomialDeriv`: `Function`
    * Returns the $k$th derivative of the polynomial whose coefficients are given by `coeffList`.

**Example**

In [28]:
coeffList = [0,1,2,3]
k = 1
x = 5
get_polynomialDeriv(coeffList, k)(x) == 2*x^1 + 2*x^0

true

## `sym_to_func(expr)`

In [152]:
function sym_to_func(expr::SymPy.Sym)
    if length(free_symbols(expr)) == 0
        result = SymPy.N(expr)
    else
        result = SymPy.lambdify(expr)
    end
    return result
end

sym_to_func (generic function with 1 method)

In [154]:
t = symbols("t")
expr = t^2+t
sym_to_func(expr)(1)

2

# Structs

## `StructDefinitionError`

A struct definition error type is the class of all errors in struct definitions.

In [29]:
struct StructDefinitionError <: Exception
    msg::String
end

## `SymLinearDifferentialOperator(symPFunctions, interval, t)`

A symbolic linear differential operator of order $n$ is encoded by an $1 \times (n+1)$ array of symbolic expressions with at most one free symbol, an interval $[a,b]$, and that free symbol.

In [30]:
struct SymLinearDifferentialOperator
    # Entries in the array should be SymPy.Sym or Number. SymPy.Sym seems to be a subtype of Number, i.e., Array{Union{Number,SymPy.Sym}} returns Array{Number}. But specifying symPFunctions as Array{Number,2} gives a MethodError when the entries are Sympy.Sym objects.
    symPFunctions::Array
    interval::Tuple{Number,Number}
    t::SymPy.Sym
    SymLinearDifferentialOperator(symPFunctions::Array, interval::Tuple{Number,Number}, t::SymPy.Sym) =
    try
        symL = new(symPFunctions, interval, t)
        check_symLinearDifferentialOperator_input(symL)
        return symL
    catch err
        throw(err)
    end
end

function check_symLinearDifferentialOperator_input(symL::SymLinearDifferentialOperator)
    symPFunctions, (a,b), t = symL.symPFunctions, symL.interval, symL.t
    for symPFunc in symPFunctions
        if isa(symPFunc, SymPy.Sym)
            if size(free_symbols(symPFunc)) != (1,) && size(free_symbols(symPFunc)) != (0,)
                throw(StructDefinitionError(:"Only one free symbol is allowed in symP_k"))
            end
        elseif !isa(symPFunc, Number)
            throw(StructDefinitionError(:"symP_k should be SymPy.Sym or Number"))
        end
    end
    return true
end

check_symLinearDifferentialOperator_input (generic function with 1 method)

**Parameters**
* `symPFunctions`: `Array` of `SymPy.Sym` or `Number`
    * Array $[symP_0, symP_1, \ldots, symP_n]$ of length $n+1$, corresponding to the symbolic linear differential operator $symL$ of order $n$ given by 
    $$symLx = symP_0x^{(n)} + symP_1x^{(n-1)} + \cdots + symP_{n-1}x^{(1)} + symP_n x.$$
* `interval`: `Tuple{Number, Number}` 
    * Tuple of two numbers $(a,b)$ corresponding to the real interval $[a,b]$ on which the symbolic differential operator $symL$ is defined.
* `t`: `SymPy.Sym` 
    * Free symbol in each entry of `symPFunctions`.

**Returns**
* `SymLinearDifferentialOperator`
    * Returns a `SymLinearDifferentialOperator` of order $n$ with attributes `symPFunctions`, `interval`, and `t`.

**Example**

In [31]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)

SymLinearDifferentialOperator(SymPy.Sym[1 t + 1 t^2 + t + 1], (0, 1), t)

## `LinearDifferentialOperator(pFunctions, interval, symL)`

A linear differential operator $L$ of order $n$ given by 
$$Lx = p_0x^{(n)} + p_1x^{(n-1)} + \cdots + p_{n-1}x^{(1)} + p_n x$$
is encoded by an $1 \times (n+1)$ array of univariate functions, an interval $[a,b]$, and its symbolic expression.

In [32]:
# symL is an attribute of L that needs to be input by the user. There are checks to make sure symL is indeed the symbolic version of L.
# Principle: Functionalities of Julia Functions >= Functionalities of SymPy. If p_k has no SymPy representation, the only consequence should be that outputs by functions that take L as arugment has no symbolic expression. E.g., we allow L.pFunctions and L.symL.pFunctions to differ.
struct LinearDifferentialOperator
    pFunctions::Array # Array of julia functions or numbers representing constant functions
    interval::Tuple{Number,Number}
    symL::SymLinearDifferentialOperator
    LinearDifferentialOperator(pFunctions::Array, interval::Tuple{Number,Number}, symL::SymLinearDifferentialOperator) =
    try
        L = new(pFunctions, interval, symL)
        check_linearDifferentialOperator_input(L)
        return L
    catch err
        throw(err)
    end
end

# Assume symFunc has only one free symbol, as required by the definition of SymLinearDifferentialOperator. 
# That is, assume the input symFunc comes from SymLinearDifferentialOperator.
function check_func_sym_equal(func::Union{Function,Number}, symFunc, interval::Tuple{Number,Number}, t::SymPy.Sym) # symFunc should be Union{SymPy.Sym, Number}, but somehow SymPy.Sym gets ignored
    (a,b) = interval
    # Randomly sample 1000 points from (a,b) and check if func and symFunc agree on them
    for i = 1:1000
        # Check endpoints
        if i == 1
            x = a
        elseif i == 2
            x = b
        else
            x = rand(Uniform(a,b), 1)[1,1]
        end
        funcEvalX = evaluate(func, x)
        if isa(symFunc, SymPy.Sym)
            symFuncEvalX = SymPy.N(subs(symFunc,t,x))
            # N() converts SymPy.Sym to Number
            # https://docs.sympy.org/latest/modules/evalf.html
            # subs() works no matter symFunc is Number or SymPy.Sym
        else
            symFuncEvalX = symFunc
        end
        tol = set_tol(funcEvalX, symFuncEvalX)
        if !isapprox(real(funcEvalX), real(symFuncEvalX); atol = real(tol)) ||
            !isapprox(imag(funcEvalX), imag(symFuncEvalX); atol = imag(tol))
            println("x = $x")
            println("symFunc = $symFunc")
            println("funcEvalX = $funcEvalX")
            println("symFuncEvalX = $symFuncEvalX")
            return false
        end
    end
    return true
end

# Check whether the inputs of L are valid.
function check_linearDifferentialOperator_input(L::LinearDifferentialOperator)
    pFunctions, (a,b), symL = L.pFunctions, L.interval, L.symL
    symPFunctions, t = symL.symPFunctions, symL.t
    # domainC = Complex(a..b, 0..0) # Domain [a,b] represented in the complex plane
    p0 = pFunctions[1]
    # p0Chebyshev = Fun(p0, a..b) # Chebysev polynomial approximation of p0 on [a,b]
    if !check_all(pFunctions, pFunc -> (isa(pFunc, Function) || isa(pFunc, Number)))
        throw(StructDefinitionError(:"p_k should be Function or Number"))
    elseif length(pFunctions) != length(symPFunctions)
        throw(StructDefinitionError(:"Number of p_k and symP_k do not match"))
    elseif (a,b) != symL.interval
        throw(StructDefinitionError(:"Intervals of L and symL do not match"))
    # # Assume p_k are in C^{n-k}. Check whether p0 vanishes on [a,b]. 
    # # roots() in IntervalRootFinding doesn't work if p0 is sth like t*im - 2*im. Neither does find_zero() in Roots.
    # # ApproxFun.roots() 
    # elseif (isa(p0, Function) && (!isempty(roots(p0Chebyshev)) || all(x->x>b, roots(p0Chebyshev)) || all(x->x<b, roots(p0Chebyshev)) || p0(a) == 0 || p0(b) == 0)) || p0 == 0 
    #     throw(StructDefinitionError(:"p0 vanishes on [a,b]"))
    elseif !all(i -> check_func_sym_equal(pFunctions[i], symPFunctions[i], (a,b), t), 1:length(pFunctions))
        # throw(StructDefinitionError(:"symP_k does not agree with p_k on [a,b]"))
        warn("symP_k does not agree with p_k on [a,b]") # Make this a warning instead of an error because the functionalities of Julia Functions may be more than those of SymPy objects; we do not want to compromise the functionalities of LinearDifferentialOperator because of the restrictions on SymPy.
    else
        return true
    end
end

check_linearDifferentialOperator_input (generic function with 1 method)

**Parameters**
* `pFunctions`: `Array` of `Function` or `Number` 
    * Array $[p_0, p_1, \ldots, p_n]$ of length $n+1$, corresponding to the linear differential operator $L$ of order $n$ given by
    $$Lx = p_0x^{(n)} + p_1x^{(n-1)} + \cdots + p_{n-1}x^{(1)} + p_n x.$$
* `interval`: `Tuple{Number, Number}`
    * Tuple of two numbers `(a, b)` corresponding to the real interval $[a,b]$ on which the differential operator $L$ is defined.
* `symL`: `SymLinearDifferentialOperator`
    * Symbolic linear differential operator corresponding to $L$.
    
**Returns**
* `LinearDifferentialOperator`
    * Returns a `LinearDifferentialOperator` of order $n$ with attributes `pFunctions`, `interval`, and `symL`.
    
**Example**

In [33]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

LinearDifferentialOperator(Function[#25 #26 #27], (0, 1), SymLinearDifferentialOperator(SymPy.Sym[1 t + 1 t^2 + t + 1], (0, 1), t))

## `VectorBoundaryForm(M, N)`

A set of homogeneous boundary conditions in vector form
$$Ux = \begin{bmatrix}U_1\\\vdots\\ U_m\end{bmatrix}x = \begin{bmatrix}\sum_{j=1}^n M_{1j}x^{(j-1)}(a) + N_{1j}x^{(j-1)}(b)\\\vdots\\ \sum_{j=1}^n M_{mj}x^{(j-1)}(a) + N_{mj}x^{(j-1)}(b)\end{bmatrix} = \begin{bmatrix}0\\\vdots\\ 0\end{bmatrix}$$
is encoded by an ordered pair of two linearly independent $m\times n$ matrices $(M, N)$ where
$$M = \begin{bmatrix}M_{11} & \cdots & M_{1n}\\ \vdots & \ddots & \vdots\\ M_{m1} & \cdots & M_{mn}\end{bmatrix},\quad N = \begin{bmatrix}N_{11} & \cdots & N_{1n}\\ \vdots & \ddots & \vdots\\ N_{m1} & \cdots & N_{mn}\end{bmatrix}.$$

In [34]:
struct VectorBoundaryForm
    M::Array # Why can't I specify Array{Number,2} without having a MethodError?
    N::Array
    VectorBoundaryForm(M::Array, N::Array) =
    try
        U = new(M, N)
        check_vectorBoundaryForm_input(U)
        return U
    catch err
        throw(err)
    end
end

# Check whether the input matrices that characterize U are valid
function check_vectorBoundaryForm_input(U::VectorBoundaryForm)
    # M, N = U.M, U.N
    # Avoid Inexact() error when taking rank()
    M = convert(Array{Complex}, U.M)
    N = convert(Array{Complex}, U.N)
    if !(check_all(U.M, x -> isa(x, Number)) && check_all(U.N, x -> isa(x, Number)))
        throw(StructDefinitionError(:"Entries of M, N should be Number"))
    elseif size(U.M) != size(U.N)
        throw(StructDefinitionError(:"M, N dimensions do not match"))
    elseif size(U.M)[1] != size(U.M)[2]
        throw(StructDefinitionError(:"M, N should be square matrices"))
    elseif rank(hcat(M, N)) != size(M)[1] # rank() throws weird "InexactError()" when taking some complex matrices
        throw(StructDefinitionError(:"Boundary operators not linearly independent"))
    else
        return true
    end
end

check_vectorBoundaryForm_input (generic function with 1 method)

**Parameters**
* `M`, `N`: `Array` of `Number`
    * Two linearly independent numeric matrices of the same dimension.

**Returns**
* `VectorBoundaryForm`
    * Returns a `VectorBoundaryForm` with attributes `M` and `N`.

**Example**

In [35]:
M = [1 0; 2 0]
N = [0 2; 0 1]
U = VectorBoundaryForm(M, N)

VectorBoundaryForm([1 0; 2 0], [0 2; 0 1])

# Construct adjoint boundary conditions

Algorithm to construct a valid adjoint boundary condition from a given (homogeneous) boundary condition based on Chapter 11 in Theory of Ordinary Differential Equations (Coddington & Levinson). The implementation uses Julia functions as main objects but supports symbolic expressions.

## `get_L(symL)`

Constructs a `LinearDifferentialOperator` from a given `SymLinearDifferentialOperator`.

In [155]:
function get_L(symL::SymLinearDifferentialOperator)
    symPFunctions, (a,b), t = symL.symPFunctions, symL.interval, symL.t
    n = length(symPFunctions)
    pFunctions = Array{Union{Function, Number}}(1,n)
    for i =1:n
        symPFunc = symPFunctions[i]
        if isa(symPFunc, SymPy.Sym) # if symPFunc is a SymPy.Sym object
            pFunc = sym_to_func(symPFunc)
#             if t in free_symbols(symPFunc)
#                 pFunc = lambdify(symPFunc)
#             else
#                 pFunc = SymPy.N(symPFunc)
#             end
        else # symPFunc could be a Number object
            pFunc = symPFunc
        end
        pFunctions[i] = pFunc
    end
    L = LinearDifferentialOperator(pFunctions, (a,b), symL)
    return L
end

get_L (generic function with 1 method)

**Parameters**
* `symL`: `SymLinearDifferentialOperator`
    * Symbolic linear differential operator to be converted.

**Returns**
* `get_L`: `LinearDifferentialOperator`
    * Returns the linear differential operator converted from `symL`.

**Example**

In [156]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

LinearDifferentialOperator(Union{Function, Number}[1 SymPy.#222 SymPy.#222], (0, 1), SymLinearDifferentialOperator(SymPy.Sym[1 t + 1 t^2 + t + 1], (0, 1), t))

## `get_URank(U)`

Computes the rank of a vector boundary form $U$ by computing the equivalent $\text{rank}(M:N)$, where $M, N$ are the matrices associated with $U$ and
$$(M:N) = \begin{bmatrix}M_{11} & \cdots & M_{1n} & N_{11} & \cdots & N_{1n}\\ \vdots & \ddots & \vdots & \vdots & \ddots & \vdots\\ M_{m1} & \cdots & M_{mn} & N_{m1} & \cdots & N_{mn}\end{bmatrix}.$$

In [38]:
function get_URank(U::VectorBoundaryForm)
    # Avoid InexactError() when taking hcat() and rank()
    M = convert(Array{Complex}, U.M)
    N = convert(Array{Complex}, U.N)
    MHcatN = hcat(M, N)
    return rank(MHcatN)
end

get_URank (generic function with 1 method)

**Parameters**
* `U`: `VectoBoundaryForm`
    * Vector boundary form whose rank is to be computed.

**Returns**
* `get_URank`: `Number`
    * Returns the rank of `U`.

**Example**

In [39]:
M = [1 0; 2 0]
N = [0 2; 0 1]
U = VectorBoundaryForm(M, N)
get_URank(U)

2

## `get_Uc(U)`

Given vector boundary form $U = \begin{bmatrix}U_1\\ \vdots\\ U_m\end{bmatrix}$ of rank $m$, finds a complementary form $U_c = \begin{bmatrix}U_{m+1}\\ \vdots\\ U_{2n}\end{bmatrix}$ of rank $2n-m$ such that $\begin{bmatrix}U_1\\ \vdots\\ U_{2n}\end{bmatrix}$ has rank $2n$.

In [40]:
function get_Uc(U::VectorBoundaryForm)
    try
        check_vectorBoundaryForm_input(U)
        n = get_URank(U)
        I = complex(eye(2*n))
        M, N = U.M, U.N
        MHcatN = hcat(M, N)
        # Avoid InexactError() when taking rank()
        mat = convert(Array{Complex}, MHcatN)
        for i = 1:(2*n)
            newMat = vcat(mat, I[i:i,:])
            newMat = convert(Array{Complex}, newMat)
            if rank(newMat) == rank(mat) + 1
                mat = newMat
            end
        end
        UcHcat = mat[(n+1):(2n),:]
        Uc = VectorBoundaryForm(UcHcat[:,1:n], UcHcat[:,(n+1):(2n)])
        return Uc
    catch err
        return err
    end
end

get_Uc (generic function with 1 method)

**Parameters**
* `U`: `VectoBoundaryForm`
    * Vector boundary form whose complementary boundary form is to be found.

**Returns**
* `get_Uc`: `VectorBoundaryForm`
    * Returns a vectory boundary form complementary to `U`.

**Example**

In [41]:
M = [1 0; 2 0]
N = [0 2; 0 1]
U = VectorBoundaryForm(M, N)
Uc = get_Uc(U)

VectorBoundaryForm(Complex[0.0+0.0im 1.0+0.0im; 0.0+0.0im 0.0+0.0im], Complex[0.0+0.0im 0.0+0.0im; 1.0+0.0im 0.0+0.0im])

## `get_H(U, Uc)`

Given a vector boundary form $U$ and a complementary vector boundary form $U_c$, constructs 
$$H = \begin{bmatrix}M&N\\ M_c & N_c\end{bmatrix},$$
where $M, N$ are the matrices associated with $U$ and $M_c, N_c$ are associated with $U_c$.

In [42]:
function get_H(U::VectorBoundaryForm, Uc::VectorBoundaryForm)
    MHcatN = hcat(convert(Array{Complex}, U.M), convert(Array{Complex}, U.N))
    McHcatNc = hcat(convert(Array{Complex}, Uc.M), convert(Array{Complex}, Uc.N))
    H = vcat(MHcatN, McHcatNc)
    return H
end

get_H (generic function with 1 method)

**Parameters**
* `U`: `VectorBoundaryForm`
    * Vector boundary form.
* `Uc`: `VectorBoundaryForm`
    * Vector boundary form complementary to `U`.

**Returns**
* `get_H`: `Array`
    * Returns the matrix $H$ defined above.

**Example**

In [43]:
M = [1 0; 2 0]
N = [0 2; 0 1]
U = VectorBoundaryForm(M, N)
Uc = get_Uc(U)
get_H(U, Uc)

4×4 Array{Complex,2}:
   1+0im      0+0im      0+0im      2+0im  
   2+0im      0+0im      0+0im      1+0im  
 0.0+0.0im  1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im  0.0+0.0im

## `get_pStringMatrix(L)`

Given a `LinearDifferentialOperator` `L` where `L.pFunctions` is the array
$$[p_0, p_1, \ldots, p_n],$$
constructs a matrix whose $(i+1)(j+1)$-entry is a string "pij" which denotes the $j$th derivative of $p_{i}$.

In [44]:
function get_pStringMatrix(L::LinearDifferentialOperator)
    if isa(L, LinearDifferentialOperator)
        pFunctions = L.pFunctions
    else
        pFunctions = L.symPFunctions
    end
    n = length(pFunctions)-1
    pStringMatrix = Array{String}(n,n)
    for i in 0:(n-1)
        for j in 0:(n-1)
            pStringMatrix[i+1,j+1] = string("p", i,j)
        end
    end
    return pStringMatrix
end

get_pStringMatrix (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `pStringMatrix` is to be constructed.

**Returns**
* `get_pStringMatrix`: `Array`
    * Returns a matrix whose $(i+1)(j+1)$-entry is a string "pij".

**Example**

In [45]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)
get_pStringMatrix(L)

2×2 Array{String,2}:
 "p00"  "p01"
 "p10"  "p11"

## `get_symPDerivMatrix(L; substitute=true)`

Given a `LinearDifferentialOperator` `L` where `L.pFunctions` is the array
$$[p_0, p_1, \ldots, p_n],$$
constructs an $n\times n$ matrix whose $(i+1)(j+1)$-entry is a symbolic expression corresponding to the $j$th derivative of $p_i$. Note that $p_n$ (or its derivaties) do not show up in the matrix.

In [46]:
# Functions with keyword arguments are defined using a semicolon in the signature.
function get_symPDerivMatrix(L::LinearDifferentialOperator; substitute = true)
    symL = L.symL
    symPFunctions, t = symL.symPFunctions, symL.t
    n = length(symPFunctions)-1
    symPDerivMatrix = Array{SymPy.Sym}(n,n)
    if substitute
        pFunctionSymbols = symPFunctions
    else
        pFunctionSymbols = [SymFunction(string("p", i-1))(t) for i in 1:n]
    end
    for i in 0:(n-1)
        for j in 0:(n-1)
            index, degree = i, j
            symP = pFunctionSymbols[index+1]
            # If symP is not a Sympy.Sym object (e.g., is a Number instead), then cannot use get_symDeriv()
            if !isa(symP, SymPy.Sym)
                if degree > 0
                    symPDeriv = 0
                else
                    symPDeriv = symP
                end
            else
                symPDeriv = get_symDeriv(symP, t, degree)
            end
            symPDerivMatrix[i+1,j+1] = symPDeriv
        end
    end
    return symPDerivMatrix
end

get_symPDerivMatrix (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `symPDerivMatrix` is to be constructed.
* `substutite*`: `Bool` 
    * Boolean indicating whether to substitute the symbolic epxression of $p_i$ (`L.symL.symPFunctions[i]`) for the $p_i$ in the generic expression $\displaystyle\frac{d^j}{dt^j} p_i(t)$.

**Returns**
* `get_symPDerivMatrix`: `Array`
    * Returns an $n\times n$ matrix whose $(i+1)(j+1)$-entry is
        * the generic symbolic expression $\displaystyle\frac{d^j}{dt^j}p_i(t)$ if `substitute = false`, or
        * the $j$th derivative of the symbolic expression of $p_i$ (`L.symL.symPFunctions[i]`) if `substitute = true`.

**Example**

In [47]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)
get_symPDerivMatrix(L; substitute = false)

2×2 Array{SymPy.Sym,2}:
 p0(t)  Derivative(p0(t), t)
 p1(t)  Derivative(p1(t), t)

In [48]:
get_symPDerivMatrix(L; substitute = true)

2×2 Array{SymPy.Sym,2}:
     1  0
 t + 1  1

## `get_pDerivMatrix(L)`

Given a `LinearDifferentialOperator` `L` where `L.pFunctions` is the array
$$[p_0, p_1, \ldots, p_n],$$
constructs an $n\times n$ matrix whose $(i+1)(j+1)$-entry is a function corresponding to the $j$th derivative of $p_i$. 

This function is not combined with `get_symPDerivMatrix` because the entries of its output are not actual `Function` objects but function-like objects converted from `SymPy.Sym` objects, whose behaviour is less clear.

In [172]:
function get_pDerivMatrix(L::LinearDifferentialOperator)
    symPDerivMatrix = get_symPDerivMatrix(L)
    n = length(L.pFunctions)-1
    pDerivMatrix = Array{Union{Function,Number}}(n,n)
    for i = 1:n
        for j = 1:n
            symPDeriv = symPDerivMatrix[i,j]
            pDerivMatrix[i,j] = sym_to_func(symPDeriv)
        end
    end
    return pDerivMatrix
end

get_pDerivMatrix (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `pDerivMatrix` is to be constructed.

**Returns**
* `get_pDerivMatrix`: `Array` of `SymPy.#`
    * Returns an $n\times n$ matrix whose $(i+1)(j+1)$-entry corresponds to the $j$th derivative of $p_i$ (`L.pFunctions[i]`).

**Example**

In [174]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

get_pDerivMatrix(L)

2×2 Array{Union{Function, Number},2}:
 1            0
  SymPy.#222  1

## `get_symUvForm(L, u, v; substitute=true)`

Create the symbolic expression for $[uv](t)$, defined as
\begin{align*}
[uv](t) := \sum_{m=1}^n\sum_{j+k=m-1}(-1)^j u^{(k)}(t)(p_{n-m}\bar{v})^{(j)}(t).
\end{align*}

In [49]:
function get_symUvForm(L::LinearDifferentialOperator, u::SymPy.Sym, v::SymPy.Sym; substitute = true)
    symL = L.symL
    symPFunctions, t = symL.symPFunctions, symL.t
    n = length(symPFunctions)-1
    if substitute
        pFunctionSymbols = symPFunctions
    else
        pFunctionSymbols = [SymFunction(string("p", i))(t) for i in 0:(n-1)]
    end
    sum = 0
    for m = 1:n
        for (j,k) in partition(m-1)
            summand = (-1)^j * get_symDeriv(u, t, k) * get_symDeriv(pFunctionSymbols[n-m+1] * conj(v), t, j)
            sum += summand
        end
    end
    sum = expand(sum)
    return sum
end

get_symUvForm (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `L.pFunctions` are to become the $p_{n-m}$ terms in $[uv](t)$.
* `u`, `v`: `SymPy.Sym`
    * Symbolic expression of $u(t)$, $v(t)$.
* `substitute*`: `Bool`
    * Boolean indicating whether to substitute the symbolic expression of $p_i$ (`L.pFunctions`[i]) for the generic expression $p_i(t)$ created using `SymFunction("pi")(t)`.

**Returns**
* `get_symUvForm`: `SymPy.Sym`
    * Returns the symbolic expression of $[uv](t)$ defined above, where $u(t)$, $v(t)$ are
        * the generic expressions $u(t)$, $v(t)$ if `substitute = false`, or
        * explicit symbolic expressions if `substitute = true`.

**Example**

In [50]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

t = symbols("t")
u, v = SymFunction("u")(t), SymFunction("v")(t) # u, v as generic functions of t
get_symUvForm(L, u, v; substitute = false)

             d /____\         ____ d                     ____        ____ d   
- p0(t)*u(t)*--\v(t)/ + p0(t)*v(t)*--(u(t)) + p1(t)*u(t)*v(t) - u(t)*v(t)*--(p
             dt                    dt                                     dt  

     
0(t))
     

In [51]:
get_symUvForm(L, u, v; substitute = true)

       ____        ____        d /____\   ____ d       
t*u(t)*v(t) + u(t)*v(t) - u(t)*--\v(t)/ + v(t)*--(u(t))
                               dt              dt      

## `get_Bjk(L, j, k; symbolic=false, substitute=true, pDerivMatrix=get_pDerivMatrix(L))`

Given a `LinearDifferentialOperator` `L` of order $n$, for $j, k \in \{1,\ldots,n\}$, computes $B_{jk}$ defined as
$$B_{jk}(t) := \sum_{\ell=j-1}^{n-k}\binom{\ell}{j-1}p^{(\ell-j+1)}_{n-k-\ell}(t)(-1)^\ell.$$

In [181]:
function get_Bjk(L::LinearDifferentialOperator, j::Int, k::Int; symbolic = false, substitute = true, pDerivMatrix = get_pDerivMatrix(L))
    n = length(L.pFunctions)-1
    if j <= 0 || j > n || k <= 0 || k > n
        throw("j, k should be in {1, ..., n}")
    end
    sum = 0
    if symbolic
        symPDerivMatrix = get_symPDerivMatrix(L; substitute = substitute)
        for l = (j-1):(n-k)
            summand = binomial(l, j-1) * symPDerivMatrix[n-k-l+1, l-j+1+1] * (-1)^l
            sum += summand
        end
    else
        if isempty(pDerivMatrix)
            throw(error("pDerivMatrix required"))
        elseif size(pDerivMatrix) != (n,n)
            throw(error("Size of pDerivMatrix should be ($n,$n)"))
        end
        for l = (j-1):(n-k)
            summand = mult_func(binomial(l, j-1) * (-1)^l, pDerivMatrix[n-k-l+1, l-j+1+1])
            sum = add_func(sum, summand)
        end
    end
    return sum
end

get_Bjk (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `L.pFunctions` are to become the $p_{n-k-l}^{l-j+1}$ in $B_{jk}(t)$.
* `j`, `k`: `Int`
    * Integers corresponding to the $j$ and $k$ in $B_{jk}$.
* `symbolic*`: `Bool`
    * Boolean indicating whether the output is symbolic.
* `substitute*`: `Bool`
    * If `symbolic = true`, boolean indicating whether to substitute the symbolic expression of $p_i$ in `L.pFunctions` for the generic expression $p_i(t)$ created using `SymFunction("pi")(t)`. If `symbolic = false`, the value of `substitute` does not matter.
* `pDerivMatrix*`: `Array`
    * If `symbolic = false`, the non-symbolic version of `symPDerivMatrix`, i.e., an $n\times n$ matrix whose $(i+1)(j+1)$-entry is the $j$th derivative of $p_i$ (`L.pFunctions[i]`) implemented as a `Function`, `Number`, or `SymPy.#`.

**Returns**
* `get_Bjk`: `SymPy.Sym`, `Function`, or `Number`
    * Returns $B_{jk}(t)$ defined above, 
        * as `Function` if `symbolic = false`, or
        * as `SymPy.Sym` object if `symbolic = true`, where the $p_i$-s are
            * generic expressions $p_i(t)$ if `substitute = false`, or
            * symbolic expressions of $p_i(t)$ (`L.symL.symPFunctions[i]`) if `substitute = true`.

**Example**

In [162]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

pDerivMatrix = [1 0; t->1+t t->1]
j, k = 1, 1
get_Bjk(L, j, k; symbolic = true, substitute = false, pDerivMatrix = pDerivMatrix)

        d        
p1(t) - --(p0(t))
        dt       

In [54]:
get_Bjk(L, j, k; symbolic = true, substitute = true, pDerivMatrix = pDerivMatrix)

t + 1

In [55]:
get_Bjk(L, j, k; symbolic = false, substitute = true, pDerivMatrix = pDerivMatrix)

(::h) (generic function with 1 method)

## `get_B(L; symbolic=false, substitute=true, pDerivMatrix=get_pDerivMatrix(L))`

Given a `LinearDifferentialOperator` `L` where `L.pFunctions` is the array
$$[p_0, p_1, \ldots, p_n],$$ 
constructs the matrix $B(t)$ whose $ij$-entry is given by
$$B_{jk}(t) := \sum_{\ell=j-1}^{n-k}\binom{\ell}{j-1}p^{(\ell-j+1)}_{n-k-\ell}(t)(-1)^\ell.$$

In [175]:
function get_B(L::LinearDifferentialOperator; symbolic = false, substitute = true, pDerivMatrix = get_pDerivMatrix(L))
    n = length(L.pFunctions)-1
    B = Array{Union{Function, Number}}(n,n)
    for j = 1:n
        for k = 1:n
            B[j,k] = get_Bjk(L, j, k; symbolic = symbolic, substitute = substitute, pDerivMatrix = pDerivMatrix)
        end
    end
    return B
end

get_B (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `L.pFunctions` are to become the $p_{n-k-l}^{l-j+1}$ in $B_{jk}(t)$.
* `symbolic*`: `Bool`
    * Boolean indicating whether the output is symbolic.
* `substitute*`: `Bool`
    * If `symbolic = true`, boolean indicating whether to substitute the symbolic expression of $p_i$ in `L.pFunctions` for the generic expression $p_i(t)$ created using `SymFunction("pi")(t)`. If `symbolic = false`, the value of `substitute` does not matter.
* `pDerivMatrix*`: `Array`
    * If `symbolic = false`, the non-symbolic version of `symPDerivMatrix`, i.e., an $n\times n$ matrix whose $(i+1)(j+1)$-entry is the $j$th derivative of $p_i$ (`L.pFunctions[i]`) implemented as a `Function` or `Number`.

**Returns**
* `get_B`: `Array` of `Function`, `SymPy.Sym`, or `Number`
    * Returns $B(t)$ defined above, where $B_{jk}(t)$ is
        * `Function` if `symbolic = false`, or
        * `SymPy.Sym` object if `symbolic = true`, where the $p_i$-s are
            * generic expressions $p_i(t)$ if `substitute = false`, or
            * symbolic expressions of $p_i(t)$ (`L.symL.symPFunctions[i]`) if `substitute = true`.

**Example**

In [178]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

pDerivMatrix = get_pDerivMatrix(L)
# pDerivMatrix = [1 0; t->1+t t->1]

get_B(L; symbolic = true, substitute = false, pDerivMatrix = pDerivMatrix)

2×2 Array{Union{Function, Number},2}:
 p1(t) - Derivative(p0(t), t)  p0(t)
                       -p0(t)      0

In [164]:
get_B(L; symbolic = true, substitute = true, pDerivMatrix = pDerivMatrix)

2×2 Array{Union{Function, Number},2}:
 t + 1  1
    -1  0

In [165]:
get_B(L; symbolic = false, substitute = false, pDerivMatrix = pDerivMatrix)

2×2 Array{Union{Function, Number},2}:
 h   h
 h  0 

## `get_BHat(L, B)`

Given a `LinearDifferentialOperator` `L` where `L.pFunctions` is the array
$$[p_0, p_1, \ldots, p_n]$$
and `L.interval` is $[a,b]$, constructs $\hat{B}$ defined as the block matrix
$$\hat{B}:=\begin{bmatrix}-B(a) & 0_n\\0_n & B(b)\end{bmatrix}.$$

In [104]:
function get_BHat(L::LinearDifferentialOperator, B::Array)
#     if check_any(B, x->isa(x, SymPy.Sym))
#         throw("Entries of B should be Function or Number")
#     end
    pFunctions, (a,b) = L.pFunctions, L.interval
    n = length(pFunctions)-1
    BHat = Array{Complex}(2n,2n)
    BEvalA = evaluate_matrix(B, a)
    BEvalB = evaluate_matrix(B, b)
    BHat[1:n,1:n] = -BEvalA
    BHat[(n+1):(2n),(n+1):(2n)] = BEvalB
    BHat[1:n, (n+1):(2n)] = 0
    BHat[(n+1):(2n), 1:n] = 0
    return BHat
end

get_BHat (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `L.pFunctions` are to become the $p_{n-k-l}^{l-j+1}$ in $B_{jk}(t)$.
* `B`: `Array` of `Number`
    * Output of `get_B(L; symbolic = false)`.
    
**Returns**
* `get_BHat`: `Array` of `Number`
    * Returns $\hat{B}$ defined above.

**Example**

In [166]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

pDerivMatrix = get_pDerivMatrix(L)
# pDerivMatrix = [1 0; t->1+t t->1]

B = get_B(L; symbolic = false, substitute = false, pDerivMatrix = pDerivMatrix)
get_BHat(L, B)

4×4 Array{Complex,2}:
 -1+0im  -1+0im   0+0im  0+0im
  1+0im   0+0im   0+0im  0+0im
  0+0im   0+0im   2+0im  1+0im
  0+0im   0+0im  -1+0im  0+0im

## `get_J(BHat, H)`

Given $\hat{B}$ and $H$, constructs $J$ defined as 
$$J:=(\hat{B}H^{-1})^\star$$
where $^*$ denotes conjugate transpose.

In [62]:
function get_J(BHat, H)
    n = size(H)[1]
    H = convert(Array{Complex}, H)
    J = (BHat * inv(H))'
    # J = convert(Array{Complex}, J)
    return J
end

get_J (generic function with 1 method)

**Parameters**
* `BHat`: `Array`
    * Output of `get_BHat()`.
* `H`: `Array`
    * Output of `get_H()`.

**Returns**
* `get_J`: `Array`
    * Returns $J$ defined above.

**Example**

In [168]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

pDerivMatrix = get_pDerivMatrix(L)
# pDerivMatrix = [1 0; t->1+t t->1]

B = get_B(L; symbolic = false, substitute = false, pDerivMatrix = pDerivMatrix)
BHat = get_BHat(L, B)

M = [1 0; 2 0]
N = [0 2; 0 1]
U = VectorBoundaryForm(M, N)
Uc = get_Uc(U)
H = get_H(U, Uc)

get_J(BHat, H)

4×4 Array{Complex,2}:
  0.333333-0.0im  -0.333333-0.0im   0.666667-0.0im   0.0-0.0im
 -0.666667-0.0im   0.666667-0.0im  -0.333333-0.0im   0.0-0.0im
      -1.0-0.0im        0.0-0.0im        0.0-0.0im   0.0-0.0im
       0.0-0.0im        0.0-0.0im        2.0-0.0im  -1.0-0.0im

## `get_adjoint(J)`

Given $J$, constructs a candidate adjoint vector boundary form $U^+$ from two matrices $P^\star$, $Q^\star$, which are the lower-left $n\times n$ submatrix of $J$, and the lower-right $n\times n$ submatrix of $J$, respectively.

In [64]:
function get_adjoint(J)
    n = convert(Int, size(J)[1]/2)
    J = convert(Array{Complex}, J)
    PStar = J[(n+1):2n,1:n]
    QStar = J[(n+1):2n, (n+1):2n]
    adjointU = VectorBoundaryForm(PStar, QStar)
    return adjointU
end

get_adjoint (generic function with 1 method)

**Parameters**
* `J`: `Array`
    * Output of `get_J`.

**Returns**
* `get_adjoint`: `VectorBoundaryForm`
    * Returns $U^+$ defined above.

**Example**

In [169]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

pDerivMatrix = get_pDerivMatrix(L)
# pDerivMatrix = [1 0; t->1+t t->1]
j, k = 1, 1

B = get_B(L; symbolic = false, substitute = false, pDerivMatrix = pDerivMatrix)
BHat = get_BHat(L, B)

M = [1 0; 2 0]
N = [0 2; 0 1]
U = VectorBoundaryForm(M, N)
Uc = get_Uc(U)
H = get_H(U, Uc)

J = get_J(BHat, H)
adjoint = get_adjoint(J)

VectorBoundaryForm(Complex[-1.0-0.0im 0.0-0.0im; 0.0-0.0im 0.0-0.0im], Complex[0.0-0.0im 0.0-0.0im; 2.0-0.0im -1.0-0.0im])

## `get_symXi(L; substitute=false, xDef=nothing)`

Given a `LinearDifferentialOperator` `L` of order $n$ in the differential equation $Lx=0$, constructs the symbolic expression of $\xi(t)$, which is defined as the vector of derivatives of $x(t)$
$$\xi(t) := \begin{bmatrix}x(t)\\ x^{(1)}(t)\\ x^{(2)}(t)\\ \vdots\\ x^{(n-1)}(t)\end{bmatrix}.$$
Note that the non-symbolic counterpart can only be constructed by hand, if needed.

In [70]:
function get_symXi(L::LinearDifferentialOperator; substitute = false, xDef = nothing)
    n = length(L.pFunctions)-1
    t = symbols("t")
    symXi = Array{SymPy.Sym}(n,1)
    if substitute
        if isa(xDef, Void)
            throw(error("Definition of x required"))
        else
            for i = 1:n
                symXi[i] = get_symDeriv(xDef,t,i-1)
            end
        end
    else
        xDef = SymFunction("x")(t)
        for i = 1:n
            symXi[i] = get_symDeriv(xDef,t,i-1)
        end
    end
    return symXi
end

get_symXi (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator in the differential equation $Lx=0$; derivatives of $x(t)$ will be entries of $\xi(t)$.
* `substitute*`: `Bool`
    * Boolean indicating whether to substitute the symbolic expression of $x(t)$ for the generic expression created using `SymFunction`.
* `xDef*`: `SymPy.Sym`
    * If `substitute = true`, symbolic expression of $x(t)$ to replace the generic expression with.

**Returns**
* `get_symXi`: `Array` of `SymPy.Sym`
    * Returns an array whose $i$th entry is
        * the generic expression $\displaystyle\frac{d^{i-1}}{dt^{i-1}}x(t)$ if `substitute = false`, or
        * the symbolic expression of the ($i-1$)th derivative of $x(t)$ if `substitute = true`.

**Example**

In [71]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

get_symXi(L; substitute = false, xDef = nothing)

2×1 Array{SymPy.Sym,2}:
                x(t)
 Derivative(x(t), t)

In [72]:
get_symXi(L; substitute = true, xDef = t^2+2t)

2×1 Array{SymPy.Sym,2}:
 t^2 + 2*t
   2*t + 2

## `evaluate_xi(L, xi, a)`

Evalutes $\xi(t)$ at $t=a$.

In [73]:
function evaluate_xi(L::LinearDifferentialOperator, xi::Array, a::Number)
    if any(xDeriv->isa(xDeriv, SymPy.Sym), xi)
        t = L.symL.t
    else
        t = nothing
    end
    n = length(xi)
    xiEvalA = Array{Number}(n,1)
    for i = 1:n
        xiEvalA[i,1] = evaluate(xi[i,1],a,t)
    end
    return xiEvalA
end

evaluate_xi (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator in the differential equation $Lx=0$; derivatives of $x(t)$ will be entries of $\xi(t)$.
* `xi`: `Array` of `Function`, `Number`, `SymPy.Sym`
    * $\xi(t)$ to be evaluated.
* `a`: `Number`
    * Value at which $\xi(t)$ is to be evaluated.

**Returns**
* `evalute_xi`: `Array` of `Number`
    * Returns $\xi(a)$.

**Example**

In [77]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

a = 1
# Symbolic xi
symXi = get_symXi(L; substitute = true, xDef = t^2+2t)
evaluate_xi(L, symXi, a)

2×1 Array{Number,2}:
 3
 4

In [76]:
# Non-symbolic xi
xi = [t->t^2+2t; t->2t+2]
evaluate_xi(L, xi, a)

2×1 Array{Number,2}:
 3
 4

## `get_boundaryCondition(L, U, xi)`

Given a `LinearDifferentialOperator` `L` and a `VectorBoundaryForm` `U`, constructs the left hand side
$$Ux = M\xi(a) + N\xi(b)$$
of the homogeneous boundary condition $Ux=0$.

In [81]:
function get_boundaryCondition(L::LinearDifferentialOperator, U::VectorBoundaryForm, xi::Array)
    # xi cannot contain types other than Function or Number
#     if !all(xDeriv->isa(xDeriv, Union{Function, Number}), xi)
#         typeXi = typeof(xi)
#         throw(error("xi of type $typeXi does not match L of type LinearDifferentialOperator"))
#     end
    (a,b) = L.interval
    M, N = U.M, U.N
    xiEvalA = evaluate_xi(L, xi, a)
    xiEvalB = evaluate_xi(L, xi, b)
    Ux = M*xiEvalA + N*xiEvalB
    return Ux
end

get_boundaryCondition (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator in the differential equation $Lx=0$; derivatives of $x(t)$ will be entries of $\xi(t)$.
* `U`: `VectorBoundaryForm`
    * Vector boundary form in the boundary condition $Ux$.
* `xi`: `Array` of `Function`, `Number`, or `SymPy.Sym`
    * Output of `get_symXi` if each entry is `SymPy.Sym`, or 
    * user input if each entry is `Function` or `Number`.

**Returns**
* `get_boundaryCondition`: `Array` of `Number`
    * Returns the $Ux$ in the homogeneous boundary condition $Ux=0$ defined above.

**Example**

In [82]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

M = [1 0; 2 0]
N = [0 2; 0 1]
U = VectorBoundaryForm(M, N)

# Symbolic xi
symXi = get_symXi(L; substitute = true, xDef = t^2+2t)
get_boundaryCondition(L, U, symXi)

2×1 Array{SymPy.Sym,2}:
 8
 4

In [83]:
# Non-symbolic xi
xi = [t->t^2+2t; t->2t+2]
get_boundaryCondition(L, U, xi)

2×1 Array{Int64,2}:
 8
 4

## `check_adjoint(L, U, adjointU, B)`

Given a boundary value problem
$$Lx = 0,\quad Ux=0$$
with linear differential operator $L$ and vector boundary form $U$, a candidate adjoint vector boundary form $U^+$, and the matrix $B$ associated with $L$, checks whether the boundary condition
$$U^+x = 0$$
is indeed adjoint to the boundary condition
$$Ux=0.$$

In [105]:
function check_adjoint(L::LinearDifferentialOperator, U::VectorBoundaryForm, adjointU::VectorBoundaryForm, B::Array)
    (a, b) = L.interval
    M, N = U.M, U.N
    P, Q = (adjointU.M)', (adjointU.N)'
    # Avoid InexactError() when taking inv()
    BEvalA = convert(Array{Complex}, evaluate_matrix(B, a))
    BEvalB = convert(Array{Complex}, evaluate_matrix(B, b))
    left = M * inv(BEvalA) * P
    right = N * inv(BEvalB) * Q
    tol = set_tolMatrix(left, right)
    return all(i -> isapprox(left[i], right[i]; atol = tol), length(left)) # Can't use == to deterimine equality because left and right are arrays of floats
end

check_adjoint (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator in the differential equation $Lx=0$.
* `U`: `VectorBoundaryForm`
    * Vector boundary form in the boundary condition $Ux=0$.
* `adjointU`: `VectorBoundaryForm`
    * Vector boundary form in the candidate adjoint boundary condition $U^+x=0$.
* `B`: `Array` of `Function`, `Number`, or `SymPy.Sym`
    * Output of `get_B(L)`.

**Returns**
* `check_adjoint`: `Bool`
    * Returns 
        * `true` if `adjointU` is indeed adjoint to `U`, or
        * `false` otherwise.

**Example**

In [170]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

M = [1 0; 2 0]
N = [0 2; 0 1]
U = VectorBoundaryForm(M, N)
Uc = get_Uc(U)

# Non-symbolic
pDerivMatrix = get_pDerivMatrix(L)
# pDerivMatrix = [1 0; t->1+t t->1]
B = get_B(L; pDerivMatrix = pDerivMatrix)
BHat = get_BHat(L, B)
H = get_H(U, Uc)
J = get_J(BHat, H)
adjointU = get_adjoint(J)

check_adjoint(L, U, adjointU, B)

true

## `construct_validAdjoint(L, U, pDerivMatrix=get_pDerivMatrix(L))`

Given a boundary value problem
$$Lx = p_0x^{(n)} + p_1x^{(n-1)} + \cdots + p_{n-1}x^{(1)} + p_n x = 0,\quad Ux=0$$
with linear differential operator $L$ and vector boundary form $U$, an $n\times n$ matrix of derivatives
$$\begin{bmatrix}p_0 & \cdots & p_0^{(n-1)}\\ \vdots & \ddots & \vdots\\ p_{n-1} & \cdots & p_{n-1}^{(n-1)}\end{bmatrix},$$
construct $U^+$ such that the boundary condition $U^+=0$ is adjoint to the original boundary condition $Ux=0$.

In [179]:
function construct_validAdjoint(L::LinearDifferentialOperator, U::VectorBoundaryForm, pDerivMatrix=get_pDerivMatrix(L))
    B = get_B(L; pDerivMatrix = pDerivMatrix)
    BHat = get_BHat(L, B)
    Uc = get_Uc(U)
    H = get_H(U, Uc)
    J = get_J(BHat, H)
    adjointU = get_adjoint(J)
    if check_adjoint(L, U, adjointU, B)
        return adjointU
    else
        throw(error("Adjoint found not valid"))
    end
end

construct_validAdjoint (generic function with 3 methods)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator in the differential equation $Lx=0$.
* `U`: `VectorBoundaryForm`
    * Vectory boundary form in the boundary condition $Ux=0$.
* `pDerivMatrix`: `Array` of `Function`, `Number`, or `SymPy.#`
    * An $n\times n$ matrix defined above, which can be
        * output of `get_pDerivMatrix` (`SymPy.#`), or
        * user input.

**Returns**
* `construct_validAdjoint`: `VectorBoundaryForm`
    * Returns a valid vector boundary form $U^+$ such that the boundary condition $U^+x=0$ is adjoint to $Ux=0$.

**Example**

In [180]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

M = [1 0; 2 0]
N = [0 2; 0 1]
U = VectorBoundaryForm(M, N)
Uc = get_Uc(U)

construct_validAdjoint(L, U)

VectorBoundaryForm(Complex[-1.0-0.0im 0.0-0.0im; 0.0-0.0im 0.0-0.0im], Complex[0.0-0.0im 0.0-0.0im; 2.0-0.0im -1.0-0.0im])

# Approximate roots of exponential polynomial

# The Fokas Transform pair